#### Author : Gautam Badri

# 1) Simple Neural Network

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_data = datasets.FashionMNIST(
    root="data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
_x, _y = None,None

for X, y in test_dataloader:
        _x = X.shape
        _y = y.shape
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
input_shape = (_x[2], _x[3])

In [ ]:
input_shape

(28, 28)

In [ ]:
train_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [ ]:
num_classes = len(train_data.classes)

In [ ]:
num_classes

10

### 1A) Create simple NN.


### 1B) Flatten x. 


### 1C) Create a single fully connected layer. 


### 1D) Do softmax before returning in the forward function.

In [ ]:
class SimpleNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=(input_shape[0]*input_shape[1]), out_features=120)
        self.relu1 = nn.ReLU()
        # self.fc2 = nn.Linear(in_features=512, out_features=256)        
        # self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(in_features= 120, out_features = num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc3(x)
        output = self.softmax(x)

        return output

### 1E) Customized loss function (not the library version)

In [ ]:
def loss_fun(y_pred, y_actual):
  v = -(y_actual * torch.log(y_pred + 0.0001))
  v = torch.sum(v)
  return v

In [ ]:
def get_optim(model, lr=1e-6):
    optim = torch.optim.SGD(model.parameters(), lr = lr)
    return optim

In [ ]:
def train_network(train_dataloder, model, optim, loss_fn, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            optim.zero_grad()
            outputs = model(inputs)
            tmp = torch.nn.functional.one_hot(labels, num_classes= 10)
            loss = loss_fn(outputs, tmp)
            loss.backward()
            optim.step()
            running_loss += loss.item()

In [ ]:
x,y = train_data[0]
model = SimpleNeuralNetwork()
y_pred = model(x)

In [ ]:
y_pred

tensor([[0.0951, 0.0874, 0.1169, 0.0895, 0.0952, 0.0941, 0.1093, 0.1140, 0.1102,
         0.0882]], grad_fn=<SoftmaxBackward0>)

In [ ]:
y_actual = y

In [ ]:
y_actual

9

In [ ]:
y_ten = torch.zeros(10)
y_ten[y_actual] = 1

In [ ]:
torch.nn.functional.one_hot(torch.tensor(y, dtype=int), num_classes= 10)
y_pred = model(x)
print(loss_fun(y_pred, y))

tensor(207.6564, grad_fn=<SumBackward0>)


### 1F) Train on training data set

In [ ]:
optim = get_optim(model, 1e-6)
train_network(train_dataloader, model, optim, loss_fun, 10)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 6.9 MB/s 


In [ ]:
from torchmetrics import Precision, Recall, F1Score, Accuracy
from torchmetrics.classification import accuracy

In [ ]:
def test_network(dataloader, model, loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            tmp = torch.nn.functional.one_hot(y, num_classes= 10)
            pred = model(X)
            test_loss += loss_fun(pred, tmp).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    accuracy1 = Accuracy()
    print('Accuracy :', accuracy1(pred,y))
    precision = Precision(average = 'macro', num_classes = 10)
    print('precision :', precision(pred,y))

    recall = Recall(average = 'macro', num_classes = 10)
    print('recall :', recall(pred,y))
    f1_score = F1Score(average = 'macro', num_classes = 10)
    print('f1_score :', f1_score(pred,y))
    return accuracy1,precision, recall, f1_score


### 1G) Test on the test data set - report accuracy, precision, recall and F1 scores.

In [ ]:
test_network(test_dataloader,model, loss_fun)

Test Error: 
 Accuracy: 51.7%, Avg loss: 130.159781 

Accuracy : tensor(0.3750)
precision : tensor(0.3433)
recall : tensor(0.3833)
f1_score : tensor(0.3293)


(Accuracy(), Precision(), Recall(), F1Score())